In [1]:
!pip install mne scipy
!pip install pyqt5

import scipy.io
import numpy as np
import matplotlib
import mne
import os
import pathlib
import pandas as pd
import shutil
from pathlib import Path

matplotlib.use('Agg')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 18.4 MB/s eta 0:00:00


In [2]:
from tensorflow.keras.models import load_model
from sklearn.svm import SVC
import pickle

from google.colab import drive
drive.mount('/content/drive')

model_save_path = '/content/drive/MyDrive/mindlock-eeg-files/model/for-train-model-1.keras'
cnn_model = load_model(model_save_path)

svm_model_path = '/content/drive/MyDrive/mindlock-eeg-files/model/for-train-svm_model.pkl'
from joblib import load
svm_model = load(svm_model_path)

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded_files = files.upload()

Saving epoch001746-epo.fif to epoch001746-epo.fif


In [4]:
# Get username input
input_username = input("Enter your username (e.g., sub001, sub002): ")

Enter your username (e.g., sub001, sub002): sub003


In [6]:
import numpy as np
import mne
from tensorflow.keras.models import load_model
from sklearn.svm import SVC
from scipy.special import softmax

# Initialize lists for predictions and confidences
predictions = []
confidences = []

# Process each uploaded file
for filename in uploaded_files.keys():
    fif_path = filename

    # Load the EEG data
    raw = mne.read_epochs(fif_path, preload=True)
    eeg_data = raw.get_data()[:64]

    # Extract features with CNN and classify with SVM
    cnn_features = cnn_model.predict(eeg_data)
    cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)

    svm_decision_scores = svm_model.decision_function(cnn_features)
    probabilities = softmax(svm_decision_scores, axis=1)
    predicted_class = np.argmax(probabilities, axis=1)[0]
    confidence_score = np.max(probabilities, axis=1)[0] * 100

    predicted_subject = f"sub{predicted_class + 1:03d}"

    predictions.append(predicted_subject)
    confidences.append(confidence_score)

    # Check if input username matches predicted subject
    if input_username == predicted_subject:
        status = "User authenticated"
    else:
        status = "User rejected"

    print(f"Predicted subject for {filename}: {predicted_subject} with {confidence_score:.2f}% confidence")
    print(f"Authentication status: {status}")

Reading /content/epoch001746-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted subject for epoch001746-epo.fif: sub003 with 67.62% confidence
Authentication status: User authenticated


In [ ]:
import numpy as np
import mne
from tensorflow.keras.models import load_model
from sklearn.svm import SVC
from scipy.special import softmax

predictions = []
confidences = []

for filename in uploaded_files.keys():
    fif_path = filename
    raw = mne.read_epochs(fif_path, preload=True)
    eeg_data = raw.get_data()[:64]
    cnn_features = cnn_model.predict(eeg_data)
    cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)

    svm_decision_scores = svm_model.decision_function(cnn_features)
    probabilities = softmax(svm_decision_scores, axis=1)
    predicted_class = np.argmax(probabilities, axis=1)[0]
    confidence_score = np.max(probabilities, axis=1)[0] * 100

    predicted_subject = f"sub{predicted_class + 1:03d}"

    predictions.append(predicted_subject)
    confidences.append(confidence_score)

    print(f"Predicted subject for {filename}: {predicted_subject} with {confidence_score:.2f}% confidence")

Reading /content/epoch052581-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted subject for epoch052581-epo.fif: sub018 with 63.75% confidence

All Predictions with Confidence Scores:
epoch052581-epo.fif: sub018 with 63.75% confidence


In [ ]:
predictions = []

for filename in uploaded_files.keys():
    fif_path = filename
    raw = mne.read_epochs(fif_path, preload=True)
    eeg_data = raw.get_data()[:64]
    cnn_features = cnn_model.predict(eeg_data)
    cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)

    svm_decision_scores = svm_model.decision_function(cnn_features)
    predicted_class = np.argmax(svm_decision_scores)
    predicted_subject = f"sub{predicted_class + 1:03d}"

    predictions.append(predicted_subject)
    print(f"Predicted subject for {filename}: {predicted_subject}")

Reading /content/epoch008434-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted subject for epoch008434-epo.fif: sub014

All Predictions:
epoch008434-epo.fif: sub014
